In [ ]:
import numpy as np
import pandas as pd
from tqdm import trange

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier

In [ ]:
def f1_macro_score(y_true, y_pred):
    return f1_score(
        y_true=y_true,
        y_pred=y_pred,
        average='macro',
    )

### load pays.csv

In [ ]:
pays = pd.read_csv('data/pays.csv')

In [ ]:
pays.head()

In [ ]:
pays.tail()

### pays analysis

In [ ]:
pays.shape

In [ ]:
pays.info()

In [ ]:
pays.describe().apply(lambda x: round(x, 2))

In [ ]:
# null values
pays.isna().sum(axis=0)

### pays features analysis

In [ ]:
for column in pays.columns:
    print(f"{column} nunique values: {pays[column].nunique()}")

In [ ]:
plt.hist(
    pays['week'], 
    edgecolor='black',
    bins=pays['week'].nunique()-1,
    align='left',
)
plt.title('week')
plt.xticks(np.arange(0, max(pays['week']) + 1, 1))
plt.grid()

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
plt.hist(pays['count'], edgecolor='black')
plt.title('count')
plt.grid()

plt.subplot(1, 3, 2)
plt.hist(pays[pays['count'] <= 500]['count'], edgecolor='black')
plt.title('count')
plt.grid()

plt.subplot(1, 3, 3)
plt.hist(pays[pays['count'] <= 50]['count'], edgecolor='black')
plt.title('count')
plt.grid()

In [ ]:
plt.hist(pays[pays['sum'] < 0]['sum'].dropna(), edgecolor='black')
plt.title('sum < 0')
plt.grid()

### load inn_info_public.csv

In [ ]:
inn_info_public = pd.read_csv('data/inn_info_public.csv')

In [ ]:
inn_info_public.head()

In [ ]:
inn_info_public.tail()

### inn_info_public analysis

In [ ]:
inn_info_public.shape

In [ ]:
inn_info_public.info()

In [ ]:
inn_info_public.describe().apply(lambda x: round(x, 2))

In [ ]:
# null values
inn_info_public.isnull().sum(axis=0)

### inn_info_public features analysis

In [ ]:
for column in inn_info_public.columns:
    print(f"{column} nunique values: {inn_info_public[column].nunique()}")

In [ ]:
inn_info_public[inn_info_public['okved2'] != -1]['okved2'].value_counts()

In [ ]:
# accuracy for the constant algorithm, which always produces the label of the largest class, is 27.8%
inn_info_public[inn_info_public['okved2'] != -1]['okved2'].value_counts(normalize=True)

In [ ]:
# train / test
inn_info_public['is_public'].value_counts()

In [ ]:
plt.figure(figsize=(10, 10))
plt.grid()
plt.xlabel('region')
plt.ylabel('okved2')
plt.scatter(
    inn_info_public[inn_info_public['okved2'] != -1][['region']],
    inn_info_public[inn_info_public['okved2'] != -1][['okved2']],
)

In [ ]:
# if a company is from region 0 or 40, then we can definitely tell its okved2
inn_info_public[inn_info_public['okved2'] != -1].pivot_table(
    index='region',
    values='okved2',
    aggfunc='nunique',
).sort_values(by='okved2')

### merge datasets

In [ ]:
graph_df = pays.merge(
    right=inn_info_public.drop('is_public', axis=1),
    left_on='hash_inn_kt',
    right_on='hash_inn',
    how='left',
).drop(['hash_inn'], axis=1).merge(
    inn_info_public.drop('is_public', axis=1),
    left_on='hash_inn_dt',
    right_on='hash_inn',
    how='left',
    suffixes=['_kt', '_dt']
).drop(['hash_inn'], axis=1)

In [ ]:
graph_df.head()

In [ ]:
graph_df.tail()

### baseline

let's try for each INN give a industry of INN with which we send most payments

In [ ]:
# straight-forward solution causes:
# "ValueError: Unstacked DataFrame is too big, causing int32 overflow"
# so we will iterate over chunks

chunks = np.linspace(pays['hash_inn_kt'].min(), pays['hash_inn_kt'].max() + 1)

inn_with_max_value = []

for i in trange(len(chunks) - 1):
    lower_inn = chunks[i]
    upper_inn = chunks[i+1]
    
    chunk = pays[(pays['hash_inn_kt'] >= lower_inn) & (pays['hash_inn_kt'] < upper_inn)]
    
    inn_with_max_value_chunk = chunk.pivot_table(
        index='hash_inn_kt',
        columns='hash_inn_dt',
        values='count',
        aggfunc='sum',
    ).idxmax(axis=1)  # take INN with max value (argmax)
    
    inn_with_max_value.append(inn_with_max_value_chunk)

In [ ]:
inn_with_max_value = pd.concat(inn_with_max_value)
inn_with_max_value.name = 'inn_with_max_value'

In [ ]:
inn_with_max_value

In [ ]:
y_true = inn_info_public.set_index(['hash_inn'])['okved2']

In [ ]:
y_true.value_counts()

In [ ]:
y_pred = pd.merge(
    left=inn_with_max_value,
    right=y_true,
    left_on='inn_with_max_value',
    right_index=True,
)['okved2']

In [ ]:
y_pred.value_counts()

In [ ]:
okved_compare = pd.merge(
    left=y_pred,
    right=y_true,
    left_index=True,
    right_index=True,
    how='inner',
    suffixes=['_pred', '_true'],
)

In [ ]:
okved_compare

In [ ]:
# filter -1 industry
filter_idx = (okved_compare['okved2_pred'] != -1) & (okved_compare['okved2_true'] != -1)

In [ ]:
accuracy = accuracy_score(
    y_true=okved_compare[filter_idx]['okved2_true'],
    y_pred=okved_compare[filter_idx]['okved2_pred'],
)

In [ ]:
# let's try for each INN give a industry of INN with which we send most payments
print(f"accuracy: {round(accuracy, 3)}")

In [ ]:
print(
    classification_report(
        y_true=okved_compare[filter_idx]['okved2_true'],
        y_pred=okved_compare[filter_idx]['okved2_pred'],
    )
)

### feature generation

In [ ]:
# the number of unique INN to whom the transaction were made
feature_1 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='hash_inn_dt',
    aggfunc='nunique',
)['hash_inn_dt']
feature_1.head()

In [ ]:
# the number of unique INN to whom the transaction were made
feature_1_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='hash_inn_dt',
    aggfunc='nunique').fillna(0)
feature_1_week.head()

In [ ]:
# the number of weeks transactions were made throughout
feature_2 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='week',
    aggfunc='nunique',
)['week']
feature_2.head()

In [ ]:
# number of transactions
feature_3 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='count',
    aggfunc='nunique',
)['count']
feature_3.head()

In [ ]:
feature_3_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='count',
    aggfunc='nunique',
).fillna(0)
feature_3_week.head()

In [ ]:
feature_4 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='count',
    aggfunc='count',
)['count']
feature_4.head()

In [ ]:
feature_4_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='count',
    aggfunc='count',
).fillna(0)
feature_4_week.head()

In [ ]:
feature_5 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='count',
    aggfunc='sum',
)['count']
feature_5.head()

In [ ]:
feature_5_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='count',
    aggfunc='sum',
).fillna(0)
feature_5_week.head()

In [ ]:
# amount of payments
feature_6 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='sum',
    aggfunc='nunique',
)['sum']
feature_6.head()

In [ ]:
feature_6_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='sum',
    aggfunc='nunique',
).fillna(0)
feature_6_week.head()

In [ ]:
feature_7 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='sum',
    aggfunc='sum',
)['sum']
feature_7.head()

In [ ]:
feature_7_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='sum',
    aggfunc='sum',
).fillna(0)
feature_7_week.head()

In [ ]:
# number of unique regions where transactions were made
feature_8 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='region_dt',
    aggfunc='nunique',
)['region_dt']
feature_8.head()

In [ ]:
feature_8_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='region_dt',
    aggfunc='nunique',
).fillna(0)
feature_8_week.head()

In [ ]:
# number of unique industries of companies where transactions were made
feature_9 = graph_df.pivot_table(
    index='hash_inn_kt',
    values='okved2_dt',
    aggfunc='nunique',
)['okved2_dt']
feature_9.head()

In [ ]:
feature_9_week = graph_df.pivot_table(
    index='hash_inn_kt',
    columns='week',
    values='okved2_dt',
    aggfunc='nunique',
).fillna(0)
feature_9_week.head()

In [ ]:
# the number of unique INN that made transactions to us
feature_10 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='hash_inn_kt',
    aggfunc='nunique',
)['hash_inn_kt']
feature_10.head()

In [ ]:
feature_10_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='hash_inn_kt',
    aggfunc='nunique',
).fillna(0)
feature_10_week.head()

In [ ]:
# the number of weeks during which transactions were made to us
feature_11 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='week',
    aggfunc='nunique',
)['week']
feature_11.head()

In [ ]:
# number of transactions
feature_12 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='count',
    aggfunc='nunique',
)['count']
feature_12.head()

In [ ]:
feature_12_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='count',
    aggfunc='nunique',
).fillna(0)
feature_12_week.head()

In [ ]:
feature_13 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='count',
    aggfunc='count',
)['count']
feature_13.head()

In [ ]:
feature_13_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='count',
    aggfunc='count',
).fillna(0)
feature_13_week.head()

In [ ]:
feature_14 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='count',
    aggfunc='sum',
)['count']
feature_14.head()

In [ ]:
feature_14_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='count',
    aggfunc='sum',
).fillna(0)
feature_14_week.head()

In [ ]:
# amount of payments
feature_15 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='sum',
    aggfunc='nunique',
)['sum']
feature_15.head()

In [ ]:
feature_15_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='sum',
    aggfunc='nunique',
).fillna(0)
feature_15_week.head()

In [ ]:
feature_16 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='sum',
    aggfunc='sum',
)['sum']
feature_16.head()

In [ ]:
feature_16_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='sum',
    aggfunc='sum',
).fillna(0)
feature_16_week.head()

In [ ]:
# the number of unique regions from which transactions were made to us
feature_17 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='region_kt',
    aggfunc='nunique',
)['region_kt']
feature_17.head()

In [ ]:
feature_17_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='region_kt',
    aggfunc='nunique',
).fillna(0)
feature_17_week.head()

In [ ]:
# the number of unique industries of companies from which transactions were made to us
feature_18 = graph_df.pivot_table(
    index='hash_inn_dt',
    values='okved2_kt',
    aggfunc='nunique',
)['okved2_kt']
feature_18.head()

In [ ]:
feature_18_week = graph_df.pivot_table(
    index='hash_inn_dt',
    columns='week',
    values='okved2_kt',
    aggfunc='nunique',
).fillna(0)
feature_18_week.head()

### combine generated features

In [ ]:
df = inn_info_public.set_index(['hash_inn']).copy()

for i, feature in enumerate([
    feature_1,
    feature_2,
    feature_3,
    feature_4,
    feature_5,
    feature_6,
    feature_7,
    feature_8,
    feature_9,
    feature_10,
    feature_11,
    feature_12,
    feature_13,
    feature_14,
    feature_15,
    feature_16,
    feature_17,
    feature_18,
]):
    feature.name = i
    df = df.merge(feature, how='left', left_index=True, right_index=True)
    df.fillna(0, inplace=True)

In [ ]:
df.head()

In [ ]:
# df_week = inn_info_public.set_index(['hash_inn']).copy()

# for feature in [
#     feature_1_week,
#     feature_2,
#     feature_3_week,
#     feature_4_week,
#     feature_5_week,
#     feature_6_week,
#     feature_7_week,
#     feature_8_week,
#     feature_9_week,
#     feature_10_week,
#     feature_11,
#     feature_12_week,
#     feature_13_week,
#     feature_14_week,
#     feature_15_week,
#     feature_16_week,
#     feature_17_week,
#     feature_18_week,
# ]:
#     df_week = df_week.merge(feature, how='left', left_index=True, right_index=True)

In [ ]:
# df_week.head()

In [ ]:
# linear independent
(df.corr() == 1).sum(axis=1)

In [ ]:
# corr
sns.heatmap(df.corr())

### train/val

In [ ]:
X = df[df['is_public'] == True].drop(['okved2', 'is_public'], axis=1)
y = df[df['is_public'] == True]['okved2']

X_test = df[df['is_public'] == False].drop(['okved2', 'is_public'], axis=1)

In [ ]:
# train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,
    shuffle=True,
    stratify=y,
    random_state=42,
)

### KNN

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.get_params()

In [ ]:
param_grid = {
    'n_neighbors': [5, 10, 15],
    'p': [1, 2],
}
scoring = make_scorer(f1_macro_score)
cv = StratifiedKFold(n_splits=3)

In [ ]:
grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring=scoring,
    cv=cv,
    verbose=1,
)

In [ ]:
%%time
grid.fit(X_train, y_train)

In [ ]:
print(f"best f1-macro: {grid.best_score_}")
print(f"best params: {grid.best_params_}")

In [ ]:
val_accuracy = accuracy_score(grid.predict(X_val), y_val)
print(f"validation accuracy: {val_accuracy}")

In [ ]:
print(
    classification_report(
        y_true=y_val,
        y_pred=grid.predict(X_val),
    )
)

### LogReg

In [ ]:
poly = PolynomialFeatures()
scaler = StandardScaler()
logreg = LogisticRegression()

model_pipe = Pipeline([
    ('poly', poly),
    ('scaler', scaler),
    ('logreg', logreg),
])

In [ ]:
model_pipe.get_params()

In [ ]:
param_grid = {
    'poly__degree': [1, 2],
    'scaler__with_mean': [False, True],
    'scaler__with_std': [False, True],
    'clf__penalty': ['l1', 'l2'],
    'clf__multi_class': ['ovr', 'multinomial'],
    'clf__C': np.linspace(0, 2, 11),
}
scoring = make_scorer(f1_macro_score)
cv = StratifiedKFold(n_splits=3)

In [ ]:
grid = RandomizedSearchCV(
    estimator=model_pipe,
    param_distributions=param_grid,
    n_iter=50,
    scoring=scoring,
    cv=cv,
    verbose=1,
)

In [ ]:
%%time
grid.fit(X_train, y_train)

In [ ]:
print(f"best f1-macro: {grid.best_score_}")
print(f"best params: {grid.best_params_}")

In [ ]:
val_accuracy = accuracy_score(grid.predict(X_val), y_val)
print(f"validation accuracy: {val_accuracy}")

In [ ]:
print(
    classification_report(
        y_true=y_val,
        y_pred=grid.predict(X_val),
    )
)

### OHE

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [ ]:
X_region_ohe = pd.DataFrame(
    data=ohe.fit_transform(X[['region']]).toarray(),
    columns=ohe.get_feature_names(),
    index=X.index,
)
X_test_region_ohe = pd.DataFrame(
    data=ohe.transform(X_test[['region']]).toarray(),
    columns=ohe.get_feature_names(),
    index=X_test.index,
)

In [ ]:
X_region_ohe.head()

In [ ]:
X_region_ohe.shape

In [ ]:
X_tree = pd.concat(
    [X.drop(['region'], axis=1), X_region_ohe],
    axis=1,
)
X_test_tree = pd.concat(
    [X_test.drop(['region'], axis=1), X_test_region_ohe],
    axis=1,
)

In [ ]:
X_tree.head()

In [ ]:
X.shape, X_tree.shape

### train/val

In [ ]:
# train_test_split
X_train_tree, X_val_tree, y_train, y_val_tree = train_test_split(
    X_tree, y,
    test_size=0.3,
    shuffle=True,
    stratify=y,
    random_state=42,
)

### RF

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.get_params()

In [ ]:
param_grid = {
    'max_depth': [None, 3, 5, 7, 9],
    'n_estimators': [50, 100, 500],
}
scoring = make_scorer(f1_macro_score)
cv = StratifiedKFold(n_splits=3)

In [ ]:
grid = GridSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    scoring=scoring,
    cv=cv,
    verbose=1,
)

In [ ]:
%%time
grid.fit(X_train, y_train)

In [ ]:
print(f"best f1-macro: {grid.best_score_}")
print(f"best params: {grid.best_params_}")

In [ ]:
val_accuracy = accuracy_score(grid.predict(X_val), y_val)
print(f"validation accuracy: {val_accuracy}")

In [ ]:
print(
    classification_report(
        y_true=y_val,
        y_pred=grid.predict(X_val),
    )
)

### XGB

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb.get_params()

In [ ]:
param_grid = {
    'max_depth': [2, 3, 5],
    'n_estimators': [50, 100, 500],
    'learning_rate': [0.1, 0.01, 0.001]
}
scoring = make_scorer(f1_macro_score)
cv = StratifiedKFold(n_splits=3)

In [ ]:
grid = GridSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    scoring=scoring,
    cv=cv,
    verbose=1,
)

In [ ]:
%%time
grid.fit(X_train, y_train)

In [ ]:
print(f"best f1-macro: {grid.best_score_}")
print(f"best params: {grid.best_params_}")

In [ ]:
val_accuracy = accuracy_score(grid.predict(X_val), y_val)
print(f"validation accuracy: {val_accuracy}")

In [ ]:
print(
    classification_report(
        y_true=y_val,
        y_pred=grid.predict(X_val),
    )
)